# **1. Determine the data set and process it**

**Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

**Load datasets**

In [ ]:
df = pd.read_csv('2019-2022_consum_electricitat_bcn.csv',sep=",")
df1 = pd.read_csv('2023-2024_consum_electricitat_bcn.csv',sep=",")
print(df.columns)

Index(['Any', 'Data', 'Codi_Postal', 'Sector_Economic', 'Tram_Horari',
       'Valor'],
      dtype='object')


**Join the two dataframes by concatenation, as they contain the same columns.**

In [ ]:
df1 = df1.iloc[1:] # Remove the first row of the consumption data for 2023-2024 as it contains the column names
df = pd.concat([df, df1], axis=0, ignore_index=True) # Concatenate the two dataframes
df_consum = df.drop(columns=['Any']) # Remove the 'Any' column as we already have that in the date

In [ ]:
# Group by 'tram_horari', 'data', and 'codi_postal', summing 'Valor'
df_agrupat = df_consum.groupby(['Data', 'Tram_Horari', 'Codi_Postal'], as_index=False).agg({'Valor': 'sum'})

print(df_agrupat)

              Data               Tram_Horari  Codi_Postal   Valor
0       2019-01-01  De 00:00:00 a 05:59:59 h         8001   68681
1       2019-01-01  De 00:00:00 a 05:59:59 h         8002  101754
2       2019-01-01  De 00:00:00 a 05:59:59 h         8003  106116
3       2019-01-01  De 00:00:00 a 05:59:59 h         8004  110622
4       2019-01-01  De 00:00:00 a 05:59:59 h         8005  113188
...            ...                       ...          ...     ...
436165  2024-10-31                 No consta         8038   13589
436166  2024-10-31                 No consta         8039   22674
436167  2024-10-31                 No consta         8040   34629
436168  2024-10-31                 No consta         8041    2322
436169  2024-10-31                 No consta         8042     724

[436170 rows x 4 columns]


We have a problem, as the time slot is missing for some values. We will solve the problem by dividing the energy demand value between the different time slots, proportionally to the value of the time slot. This way, we will no longer have samples without a time slot.


In [ ]:
# Filter the rows where 'Tram_Horari' is 'No consta'
df_no_tram = df_agrupat[df_agrupat['Tram_Horari'] == 'No consta']

# Filter the rows with assigned time slots
df_amb_tram = df_agrupat[df_agrupat['Tram_Horari'] != 'No consta']

# Distribute the values of 'df_no_tram' proportionally across the time slots
def repartir_valor(fila_no_tram):
    data = fila_no_tram['Data']
    codi_postal = fila_no_tram['Codi_Postal']
    valor_sense_tram = fila_no_tram['Valor']

    # Filter the time slots for that date and postal code
    franjes_disponibles = df_amb_tram[(df_amb_tram['Data'] == data) & (df_amb_tram['Codi_Postal'] == codi_postal)]

    # Calculate the sum of the values for the available time slots
    suma_franjes = franjes_disponibles['Valor'].sum()

    if suma_franjes > 0:
        # Calculate the proportion and distribute
        proporcio = franjes_disponibles['Valor'] / suma_franjes
        repartiment = proporcio * valor_sense_tram

        # Add the distributed value to the existing time slots
        df_amb_tram.loc[df_amb_tram.index.isin(franjes_disponibles.index), 'Valor'] += repartiment

# Apply the distribution function to each row of df_no_tram
df_no_tram.apply(repartir_valor, axis=1)

# Combine the rows with the distributed values and the original rows
df_final = pd.concat([df_amb_tram, df_no_tram])

# Remove the rows where 'Tram_Horari' is 'No consta', as the value has already been distributed
df_final = df_final[df_final['Tram_Horari'] != 'No consta']

print(df_final)

              Data               Tram_Horari  Codi_Postal          Valor
0       2019-01-01  De 00:00:00 a 05:59:59 h         8001   71768.816539
1       2019-01-01  De 00:00:00 a 05:59:59 h         8002  105525.771005
2       2019-01-01  De 00:00:00 a 05:59:59 h         8003  109681.449902
3       2019-01-01  De 00:00:00 a 05:59:59 h         8004  114701.832375
4       2019-01-01  De 00:00:00 a 05:59:59 h         8005  117194.598557
...            ...                       ...          ...            ...
436123  2024-10-31  De 18:00:00 a 23:59:59 h         8038  141482.554850
436124  2024-10-31  De 18:00:00 a 23:59:59 h         8039  153705.681040
436125  2024-10-31  De 18:00:00 a 23:59:59 h         8040  244330.340099
436126  2024-10-31  De 18:00:00 a 23:59:59 h         8041   68883.106750
436127  2024-10-31  De 18:00:00 a 23:59:59 h         8042   39368.640974

[348936 rows x 4 columns]


Review if we have solved the **problem**


In [ ]:
print('Initial dataframe:') # Tram Horari with "no consta" values
print(df_agrupat[(df_agrupat['Data']=="2021-04-18") & (df_agrupat['Codi_Postal']==8001)])

print('-----------------------------------------------------------------------')

print('Final dataframe:') # Tram Horari without "no consta" values
print(df_final[(df_final['Data']=="2021-04-18") & (df_final['Codi_Postal']==8001)])

Load CSVs with data that provide more information to the initial dataset

In [ ]:
# Import Calendar (weekday and whether it's considered a holiday or not)
df = pd.read_csv('2019-2024_Calendari.csv', sep=";")
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")
df = df.rename(columns={'Data': 'Date'})

# Import IPI (Industrial Production Index can have a big impact on energy consumption)
df1 = pd.read_csv('tasa_interanual_del_indice_de_produccion_industrial_en_cataluña.csv', sep=";")
df1 = df1.dropna(subset=['Periodo'])

# Convert 'Periodo' to a numeric month for merging purposes.
month_mapping = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5,
    'Junio': 6, 'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10,
    'Noviembre': 11, 'Diciembre': 12
}
df1['Periodo'] = df1['Periodo'].map(month_mapping)

# Create a new column from the year and period, to match the format of the other df
df1['Date'] = pd.to_datetime(df1['Año'].astype(str) + '-' + df1['Periodo'].astype(str) + '-01') # Assign day 1, it's not relevant
df1 = df1.drop(['Año', 'Periodo'], axis=1) # Remove columns that are no longer informative
df1 = df1[['Date', 'Tasa interanual del IPI']] # Not necessary, as the other columns were already dropped, it's to ensure the result

# Display the result from the CSVs
print(df.head()) # Calendar
print("------------------------------------")
print(df1.head()) # IPI

# Merge
# Ensure the format of the data
df['Date'] = pd.to_datetime(df['Date'])
df1['Date'] = pd.to_datetime(df1['Date'])

df['Year_Month'] = df['Date'].dt.to_period('M')  # To get just the year and month
df1['Year_Month'] = df1['Date'].dt.to_period('M')  # To get just the year and month

# Merge the two dataframes based on the 'Year_Month' column
df2 = pd.merge(df, df1, on='Year_Month', how='left')
df2 = df2.drop(['Date_y', 'Year_Month'], axis=1)
df2.rename(columns={'Date_x': 'Date'}, inplace=True)

print("--------------------------------------------------------")
print(df2.head())

We will connect to a weather API

In [ ]:
pip install openmeteo-requests

In [ ]:
pip install requests-cache retry-requests numpy pandas

In [ ]:
# API
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.39978186013002,
	"longitude": 2.151839058933601,
	"start_date": "2019-01-01",
	"end_date": "2024-12-24",
	"hourly": ["temperature_2m", "apparent_temperature", "rain", "wind_speed_10m", "is_day", "sunshine_duration", "direct_radiation",'dew_point_2m'],
	"temporal_resolution": "hourly_6"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
hourly_is_day = hourly.Variables(4).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(5).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(6).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["rain"] = hourly_rain
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["dew_point_2m"] = hourly_dew_point_2m


hourly_dataframe = pd.DataFrame(data = hourly_data)

# Convert the 'date' column to datetime type for later processing
hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'])

# Define a function to determine the time slot
def determinar_franja_horaria(date):
    hora = date.hour
    if 0 <= hora < 6:
        return "From 00:00:00 to 05:59:59"
    elif 6 <= hora < 12:
        return "From 06:00:00 to 11:59:59"
    elif 12 <= hora < 18:
        return "From 12:00:00 to 17:59:59"
    elif 18 <= hora < 24:
        return "From 18:00:00 to 23:59:59"

# Apply the function to create the time slot column
hourly_dataframe["t_Horari"] = hourly_dataframe["date"].apply(determinar_franja_horaria)

# Format the date to the 'YYYY-MM-DD' format, so we have a common column for the join
hourly_dataframe["date"] = hourly_dataframe["date"].dt.strftime("%Y-%m-%d")

print(hourly_dataframe.head())

Join with electricity consumption data, created previously, to later analyze which features have a greater impact on consumption,discriminating the insignificant ones and processing the data


In [ ]:
# Merge the consumption dataframe with the weather dataframe, it must match both the time slot and the date
df_conjunta1 = pd.merge(
    df_final,
    hourly_dataframe,
    left_on=["Data", "Tram_Horari"],
    right_on=["date", "t_Horari"],
    how="inner"
)

# Merge the consumption and weather dataframe with the calendar and IPI dataframe
df_conjunta1['date'] = pd.to_datetime(df_conjunta1['date'], errors='coerce') # Ensure the correct data format
df2['Date'] = pd.to_datetime(df2['Date'], errors='coerce') # Ensure the correct data format

df_conjunta2 = pd.merge(
    df_conjunta1,
    df2,
    left_on="date",
    right_on="Date",
    how="inner"
)

# Order the remaining columns
df_conjunta3 = df_conjunta2[['date','Tram_Horari','Codi_Postal', 'Valor', 'temperature_2m','apparent_temperature','rain','wind_speed_10m','is_day','sunshine_duration','direct_radiation','Dia_Setmana','Festiu','Tasa interanual del IPI','dew_point_2m']]

# Process the columns as they are needed for further use
df_conjunta3['rain'] = df_conjunta3['rain'].astype(bool)
df_conjunta3['is_day'] = df_conjunta3['is_day'].astype(bool)
df_conjunta3['Festiu'] = df_conjunta3['Festiu'].astype(bool)
df_conjunta3['Tasa interanual del IPI'] = df_conjunta3['Tasa interanual del IPI'].str.replace(',', '.').astype(float)
df_conjunta3['date'] = pd.to_datetime(df_conjunta3['date'])

# Function to convert the "Tram_Horari" string to a categorical value
def determinar_franja_horaria(tram_horari):
    if "De 00:00:00 a 05:59:59 h" in tram_horari:
        return 1
    elif "De 06:00:00 a 11:59:59 h" in tram_horari:
        return 2
    elif "De 12:00:00 a 17:59:59 h" in tram_horari:
        return 3
    elif "De 18:00:00 a 23:59:59 h" in tram_horari:
        return 4
    else:
        return None

df_conjunta4 = df_conjunta3 # To avoid modifying the original variable

# Apply the function to create the time slot column
df_conjunta4['Tram_Horari'] = df_conjunta3['Tram_Horari'].apply(determinar_franja_horaria)

print(df_conjunta3.head())

In [ ]:
# Function to convert the "Dia_Setmana" string to categorical
df_conjunta5 = df_conjunta3  # To avoid modifying the original variable
df_conjunta5['Dia_Setmana'] = df_conjunta5['Dia_Setmana'].astype(str)  # Check the data format

def determinar_dia_setmana(Dia_Setmana):
    if "Dilluns" in Dia_Setmana:
        return 1
    elif "Dimarts" in Dia_Setmana:
        return 2
    elif "Dimecres" in Dia_Setmana:
        return 3
    elif "Dijous" in Dia_Setmana:
        return 4
    elif "Divendres" in Dia_Setmana:
        return 5
    elif "Dissabte" in Dia_Setmana:
        return 6
    elif "Diumenge" in Dia_Setmana:
        return 7
    else:
        return None

# Apply the function to create the weekday column
df_conjunta5['Dia_Setmana'] = df_conjunta5['Dia_Setmana'].apply(determinar_dia_setmana)

# Modify the format of the 'date' column to extract more information, as the moment has a large impact.
df_conjunta6 = df_conjunta5  # To avoid modifying the original variable
df_conjunta6['Any'] = df_conjunta6['date'].dt.year
df_conjunta6['Mes'] = df_conjunta6['date'].dt.month
df_conjunta6['Dia'] = df_conjunta6['date'].dt.day

# Order the columns, only keep the ones that provide information
df_conjunta6 = df_conjunta6[['Any', 'Mes', 'Dia', 'Tram_Horari', 'Codi_Postal', 'Valor', 'temperature_2m', 'apparent_temperature', 'rain', 'wind_speed_10m', 'is_day', 'sunshine_duration', 'direct_radiation', 'Dia_Setmana', 'Festiu', 'Tasa interanual del IPI', 'dew_point_2m']]
print(df_conjunta6.columns)

Handle the outliers that introduce noise into our data.

In [ ]:
plt.figure(figsize=(8, 3))
sns.boxplot(x=df_conjunta6['Valor'])
plt.title('Boxplot of the Value')
plt.show()

# Calculation of IQR (Interquartile Range)
Q1 = df_conjunta6['Valor'].quantile(0.25)
Q3 = df_conjunta6['Valor'].quantile(0.75)
IQR = Q3 - Q1

# Define the limits for outliers
lower_bound = Q1 - 1.5 * IQR # Not necessary as we don't have values lower than 0
upper_bound = Q3 + 1.5 * IQR

outliers = df_conjunta6[(df_conjunta6['Valor'] > upper_bound)] # Filter the outliers from the original dataset

# Display the outliers
print("------------------------------------------------------------------------------------")
print("25% of the data: ", Q1)
print("75% of the data: ", Q3)
print("Upper Bound: ", upper_bound)
print("Outliers: ", outliers['Valor'].count())
print("Total Dataset: ", df_conjunta6['Valor'].count())
print("Using ", 100 - (outliers['Valor'].count() / df_conjunta6['Valor'].count()) * 100, "% of the original dataset")

Eliminate outliers that are outside the noise contained in the energy demand and do not represent the norm, while keeping some of the atypical values that could be noise associated with the phenomenon.

In [ ]:
plt.figure(figsize=(8, 3))
sns.boxplot(x=outliers['Valor'])
plt.title('Boxplot of the outliers')
plt.show()

Q1 = outliers['Valor'].quantile(0.25)
Q3 = outliers['Valor'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
print("Upper Bound: ", upper_bound)

outliers = outliers[(outliers['Valor'] > upper_bound)]

df_cleaned = df_conjunta6[(df_conjunta6['Valor'] <= upper_bound)]
deleted = df_conjunta6[(df_conjunta6['Valor'] > upper_bound)]

print("Deleted Dataset: ", deleted['Valor'].count())
print("Total Dataset: ", df_cleaned['Valor'].count())
print("Using ", 100 - (deleted['Valor'].count() / df_conjunta6['Valor'].count()) * 100, "% of the original dataset")

Create a .csv with the dataframe df_cleaned

In [ ]:
df_cleaned.to_csv("dataframe_final.csv", index=False)

In [ ]:
print(df_cleaned.describe())